# Extended Kalman Filter localization example
We are considering a robot with differential traction with a rotary sensor on its top. It detects beacons and obtain a measure of its distance and angle to the robot refential frame.
Whenever we observe a beacon there is an error in the estimated angle and an error in estimating the distance. We assume that the standard deviation of the distance error increases proportionally with distance.

This work is inspired on "Laboratorial Work 4 - Robot localization based on beacons" from the course "Sistemas Robóticos Autónomos" at FEUP.
More specifically we are talking about a soccer robot in a small soccer field with the beacons in the four corner. The robot uses first a rotative of a camera and then an omnidirectional camera in the second part. 

## 1. Extended Kalman Filter implementation in robot simulator
We start by importing numpy

In [1]:
import numpy as np

and setting the initial state of the robot at

In [2]:
x=np.array([2, -2, 0])
x

array([ 2, -2,  0])

Differently from MATLAB at this stage does not make sense to distinguish from "horizontal" or "vertical" vectors.

We are going to simulate

In [3]:
N=300

time intervals, each of lenght

In [4]:
dt=0.040

Now we generate the control signals of the robot for each time interval

In [5]:
v_t = np.random.normal(0,1,N) + 1 #linear velocity
omega_t = np.random.normal(0,1,N)*0.5 + 0.5 #angular velocity

and we group them in

In [6]:
u_t = np.array([v_t,omega_t])
u_t.shape

(2, 300)

We set the matrices of covariances...

In [7]:
P = np.eye(3)*1e-3 # covariance robot belief from prediction update
Q = np.array([[0.1**2, 0], [0, 0.05**2]]) # covariance belief exteroceptive measurments

Standard deviation of the error of measurment for the beacons

In [8]:
sdv_dist_1m = 0.05
sdv_ang = 0.01

We set the location of the first two beacons in the map

In [9]:
xb1 = 5
yb1 = -2.5
xb2 = 5
yb2 = 2.5

The initial value for the extimated/predicted state/position of the robot is

In [10]:
x_e = np.array([2.5, -2.5, 0])
xr_e = x_e[0] 
yr_e = x_e[1]
theta_r_e = x_e[2]

Initialize the variables to record the simulation (probably this is not the smartest way, I know that...)

In [11]:
x_t = np.array([])
x_e_t = np.array([])

### Simulation

What's this?
TODO: add a cool latex equation here representing the multi-line comment below

In [39]:
from math import sin, cos, sqrt, atan2
def F(t,y,u):
    '''
    Simulation of a mobile robot
    State vector:   X(t)=[displacement along x axis - x(t)
                          displacement along y axis - y(t)
                          angular position - a(t)]
                          
                    u(t)=[linear velocity - v(t)
                          angular velocity - w(t)]
    Equations: dx(t)/dt = v(t)*cos(a(t))
               dy(t)/dt = v(t)*sin(a(t))
               da(t)/dt = w(t)
    '''
    # return dy/dt = f(t,y).
    return np.array([u[0]*cos(y[2]), u[0]*sin(y[2]), u[1]])

In the simulation we need to solve a differential equation, so we will need

In [13]:
from scipy.integrate import solve_ivp

Initialize counter to change from one beacon to the other

In [34]:
p_count = 0

Here we go...

In [54]:
for i in range(0, N):
    # record the evolution of the states
    x_t = np.concatenate((x_t, x), axis = 0)
    x_e_t = np.concatenate((x_e_t, x_e), axis = 0)

    # simulation of the robot
    v = u_t[0, i]
    omega = u_t[1, i]
    TSPAN = [0, dt]
    sol = solve_ivp(lambda t, y: F(t,y,u_t[:,i]), TSPAN, x, rtol = 1000, atol = 1000)

    x = sol.y[:,-1]
    xr = x[0]
    yr = x[1]
    theta_r = x[2]

    # update of the extimation of the state of the robot X(k+1) = f(X(k),U)
    xr_e = x_e[0] 
    yr_e = x_e[1] 
    theta_r_e = x_e[2]

    xr_e = xr_e + v * cos(theta_r_e + omega * dt / 2) * dt 
    yr_e = yr_e + v * sin(theta_r_e + omega * dt / 2) * dt 
    theta_r_e = theta_r_e + omega * dt

    x_e=np.array([xr_e, yr_e, theta_r_e]) # maybe transpose?

    # calculate grad_f_X=df/dX 
    grad_f_X = np.array([[1, 0, -v*dt*sin(theta_r_e+omega*dt/2)],
                         [0, 1, v*dt*cos(theta_r_e+omega*dt/2)],
                         [0, 0, 1]])

    # calculate grad_f_U=df/dU
    grad_f_U = np.array([[cos(theta_r_e+omega*dt/2)*dt, -v*dt*0.5*sin(theta_r_e+omega*dt/2)],
                         [sin(theta_r_e+omega*dt/2)*dt, v*dt*0.5*cos(theta_r_e+omega*dt/2)],
                         [0, 1]])

    # update covariance
    P = np.dot(grad_f_X, np.dot(P, np.transpose(grad_f_X))) + np.dot(grad_f_U, np.dot(Q, np.transpose(grad_f_U)))

    # counter to change from one beacon to the other
    p_count = p_count + 1

    if p_count==100:
        p_count=0 

    if p_count <= 50: # beacon 1
        # simulation of the observed measures
        distp = sqrt((xb1 - xr)**2 + (yb1 - yr)**2)
        distp_measured = distp + np.random.normal(0,1) * sdv_dist_1m * distp
        theta_p = atan2(yb1 - yr, xb1 - xr) - theta_r
        theta_p_measured = theta_p + np.random.normal(0,1) * sdv_ang
        z = np.array([distp_measured, theta_p_measured])

        # expected values for the measures taking into account the state of the robot z = h(X)
        distp_e = sqrt((xb1 - xr_e)**2 + (yb1 - yr_e)**2)
        theta_p_e = atan2(yb1 - yr_e, xb1 - xr_e) - theta_r_e
        z_e = np.array([distp_e, theta_p_e])

        # covariance matrix of the measures noise
        R = np.array([[(distp_e*sdv_dist_1m)**2, 0],
                      [0, sdv_ang**2]])

        # calculation of dH/dX
        grad_h_X  = np.array([[-(xb1-xr_e)/distp_e, -(yb1-yr_e)/distp_e, 0],
                              [(yb1-yr_e)/(distp_e**2), -(xb1-xr_e)/(distp_e**2), -1]])           
    else: # beacon 2
        # simulation of the observed measures
        distp = sqrt((xb2 - xr)**2 + (yb2 - yr)**2)
        distp_measured = distp + np.random.normal(0,1) * sdv_dist_1m * distp
        theta_p = atan2(yb2 - yr, xb2 - xr) - theta_r
        theta_p_measured = theta_p + np.random.normal(0,1) * sdv_ang
        z = np.array([distp_measured, theta_p_measured])

        # expected values for the measures taking into account the state of the robot z = h(X)
        distp_e = sqrt((xb2 - xr_e)**2 + (yb2 - yr_e)**2)
        theta_p_e = atan2(yb2 - yr_e, xb2 - xr_e) - theta_r_e
        z_e = np.array([distp_e, theta_p_e])

        # covariance matrix of the measures noise
        R = np.array([[(distp_e*sdv_dist_1m)**2, 0],
                      [0, sdv_ang**2]])

        # calculation of dH/dX
        grad_h_X  = np.array([[-(xb2-xr_e)/distp_e, -(yb2-yr_e)/distp_e, 0],
                              [(yb2-yr_e)/(distp_e**2), -(xb2-xr_e)/(distp_e**2), -1]])


    # Kalman gain
    k = np.dot(P,np.dot(np.transpose(grad_h_X), np.linalg.inv(np.dot(grad_h_X, np.dot(P, np.transpose(grad_h_X)))) + R));

    # covariance update
    P = np.dot(np.eye(3) - np.dot(k, grad_h_X), P)

    # update expected state
    x_e = x_e + np.dot(k, z - z_e)

### Visualize the results
Let's see what we got...

In [57]:
from matplotlib.pyplot import plot
L = 0.01
plot(x_t[0,:],x_t[1,:] ,'o',
     #x_t(1,:)+np.dot(L, np.cos(x_t(3,:))),   x_t(2,:)+np.dot(L, np.sin(x_t(3,:))) ,'.',
     #x_e_t(1,:),   x_e_t(2,:) ,'o',
     #x_e_t(1,:)+np.dot(L, np.cos(x_e_t(3,:))),   x_e_t(2,:)+np.dot(L, np.sin(x_e_t(3,:))) ,'.' ,
     xp1,yp1 ,'p', xp2,yp2, 'h')
#legend(   'xy',          'dir', 'xy_e',          'dir_e', 'Pole 1',  'Pole 2' ), 

IndexError: too many indices for array

## 2. Implementation considering only angle or distance measure
Content

## 3. Four beacons simultaneously
Content

## 4. Four beacons: only angle or distance
Content

## 5. Observations in coordinates of the robot referential frame
Content